In [1]:
import numpy as np
import pandas as pd


# Question 1

## Energy Indicators

In [2]:
# Header -> sets the "starting row" where the reader begins to read. Row (0-indexed) to use for the column labels of the parsed DataFrame. 
    ## If a list of integers is passed those row positions will be combined into a MultiIndex. 
# Names -> List of column names to use. If file contains no header row, then you should explicitly pass header=None.
# usecols -> If str, then indicates comma separated list of Excel column letters and column ranges (e.g. “A:E” or “A,C,E:F”). Ranges are inclusive of both sides.

# dropna deletes all rows with nans

energy=pd.read_excel("datasets\Energy Indicators.xls",
                     header=[17],
                     names=['Country', 'Energy Supply(PJ)', 'Energy Supply per Capita(GJ)', '% Renewable'],
                     usecols="C:F")
energy.dropna(inplace=True)

In [3]:
# transforming PJ to GJ through apply and lambda. apply applies the lambda to every row of the resulting series
energy['Energy Supply(PJ)'] = energy['Energy Supply(PJ)'].apply(lambda x: x*1000000)
energy.head(20)

,Country,Energy Supply(PJ),Energy Supply per Capita(GJ),% Renewable
0,Afghanistan,321000000,10,78.669280
1,Albania,102000000,35,100.000000
2,Algeria,1959000000,51,0.551010
3,American Samoa,.................................................,...,0.641026
4,Andorra,9000000,121,88.695650
5,Angola,642000000,27,70.909090
6,Anguilla,2000000,136,0.000000
7,Antigua and Barbuda,8000000,84,0.000000
8,Argentina,3378000000,79,24.064520
9,Armenia,143000000,48,28.236060


In [4]:
# transforming empty values ("...") to nans
# using relative positioning energy.iloc[3].loc["Energy Supply(PJ)"] to replace unfilled values
energy.replace(energy.iloc[3].loc["Energy Supply(PJ)"], np.nan, inplace=True)

In [5]:
# replacing column names directly on attribute to better fit new data
energy.columns = ['Country', 'Energy Supply(GJ)', 'Energy Supply per Capita(GJ)', '% Renewable']
energy.head()

,Country,Energy Supply(GJ),Energy Supply per Capita(GJ),% Renewable
0,Afghanistan,3.210000e+08,10,78.669280
1,Albania,1.020000e+08,35,100.000000
2,Algeria,1.959000e+09,51,0.551010
3,American Samoa,NaN,...,0.641026
4,Andorra,9.000000e+06,121,88.695650


In [6]:
# cleaning countries with parenthesis

# finding the iloc of the offending countries
# this returns a dataframe with the country AND the "extra part" with proper labelled columns
country_names = energy["Country"].str.extract("(?P<Country>.*)(?P<Extra>\(.*\))").dropna()

#extracting the index loc from the DF as a list
locs = list(country_names.index)

# extracting proper names
names = list(country_names["Country"])

#iterating through index locs list to replace the value
i = 0
for loc in locs:
    energy.loc[loc,["Country"]] = names[i]
    i +=1


In [7]:
# verifying elimination fo paranthesis
energy.loc[[24,70,98,133,183,221,222]]

,Country,Energy Supply(GJ),Energy Supply per Capita(GJ),% Renewable
24,Bolivia,3.360000e+08,32,31.477120
70,Falkland Islands,1.000000e+06,258,33.333330
98,Iran,9.172000e+09,119,5.707721
133,Micronesia,2.000000e+06,21,2.941176
183,Sint Maarten,1.200000e+07,327,0.000000
221,Venezuela,2.871000e+09,95,67.834520
222,Viet Nam,2.554000e+09,28,45.321520


In [8]:
# cleaning countries with numbers in the name

# create a dataframe with country and the digit separated
clean_names = energy["Country"].str.extract("(?P<Country>.*\D)(?P<Digit>\d{1,4})").dropna()

# extracting relevant index locations as a list
locs = list(clean_names.index)

# extracting proper country names as a list
names2 = list(clean_names["Country"])

# iterating through index loc to correct
i=0
for loc in locs:
    energy.loc[loc,["Country"]] = names2[i]
    i += 1

In [9]:
energy.loc[43].iloc[0]

'China, Hong Kong Special Administrative Region'

In [10]:
# replacing countries names as per guidance
# passing a dictionary into replace directly transforms a current value (key) into a new value (value)
energy.replace({"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"}, inplace=True)

In [11]:
energy.loc[43].iloc[0]

'Hong Kong'

In [12]:
# cleaning spaces on a delinquent row!
energy.loc[98,["Country"]] = "Iran"

## GDP Data

In [13]:
# Header -> sets the "starting row" where the reader begins to read. Row (0-indexed) to use for the column labels of the parsed DataFrame. 
    ## If a list of integers is passed those row positions will be combined into a MultiIndex. 

gdp = pd.read_csv("datasets\world_bank.csv", header=4)


In [14]:
# replacing incorrectly named countries.
gdp.replace({"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"}, inplace=True)

# renaming one column for consistency
gdp.rename(columns={"Country Name":"Country"},inplace=True)
gdp.head()

,Country,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
0,Aruba,ABW,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.467704e+09,NaN,NaN,NaN,NaN,NaN
1,Andorra,AND,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,4.018196e+09,4.021331e+09,3.675728e+09,3.535389e+09,3.346317e+09,3.185605e+09,3.129538e+09,3.127550e+09,NaN,NaN
2,Afghanistan,AFG,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.030523e+10,1.172119e+10,1.214448e+10,1.469733e+10,1.593680e+10,1.691113e+10,1.935220e+10,1.973134e+10,1.999032e+10,2.029415e+10
3,Angola,AGO,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,5.581103e+10,6.842044e+10,7.787420e+10,7.975320e+10,8.247091e+10,8.570262e+10,9.012096e+10,9.626143e+10,1.008863e+11,1.039106e+11
4,Albania,ALB,GDP at market prices (constant 2010 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,9.771760e+09,1.034829e+10,1.112752e+10,1.150029e+10,1.192695e+10,1.223109e+10,1.240477e+10,1.254247e+10,1.279331e+10,1.312082e+10


In [15]:
# verifying proper replacement
gdp["Country"].str.extract("(?P<Country>South Korea|Iran|Hong Kong)").dropna()

,Country
93,Hong Kong
109,Iran
123,South Korea


## Sciamgo Journal

In [16]:
scimen = pd.read_excel("datasets\scimagojr-3.xlsx")
scimen.head(15)

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index
0,1,China,127050,126767,597237,411683,4.70,138
1,2,United States,96661,94747,792274,265436,8.20,230
2,3,Japan,30504,30287,223024,61554,7.31,134
3,4,United Kingdom,20944,20357,206091,37874,9.84,139
4,5,Russian Federation,18534,18301,34266,12422,1.85,57
5,6,Canada,17899,17620,215003,40930,12.01,149
6,7,Germany,17027,16831,140566,27426,8.26,126
7,8,India,15005,14841,128763,37209,8.58,115
8,9,France,13153,12973,130632,28601,9.93,114
9,10,South Korea,11983,11923,114675,22595,9.57,104


In [17]:
df = pd.merge(scimen.loc[0:14],energy,how="inner",on="Country")
df

,Rank,Country,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply(GJ),Energy Supply per Capita(GJ),% Renewable
0,1,China,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910
1,2,United States,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980
2,3,Japan,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.232820
3,4,United Kingdom,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.600470
4,5,Russian Federation,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680
5,6,Canada,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.945430
6,7,Germany,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.901530
7,8,India,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080
8,9,France,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.020280
9,10,South Korea,11983,11923,114675,22595,9.57,104,1.100700e+10,221,2.279353


In [18]:
df = pd.merge(df,gdp.loc[:,["Country",'2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']],how="inner",on="Country")
df.set_index("Country",inplace=True)

# Question 2

In [19]:
'''The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

This function should return a single number. AS PER DISCS, THIS IS THE DIF BETWEEN FULL OUTER AND FULL INNER'''

'The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?\n\nThis function should return a single number. AS PER DISCS, THIS IS THE DIF BETWEEN FULL OUTER AND FULL INNER'

In [20]:
len(df)

15

In [21]:
# full outer join vs the join we did vs instructions
outer_df = pd.merge(pd.merge(energy,gdp,how="outer",on="Country"),scimen,how="outer",on="Country")
len(outer_df) - len(df)

306

# Question 3

In [22]:
'''What are the top 15 countries for average GDP over the last 10 years?

This function should return a Series named avgGDP with 15 countries and their average GDP sorted in descending order.'''

'What are the top 15 countries for average GDP over the last 10 years?\n\nThis function should return a Series named avgGDP with 15 countries and their average GDP sorted in descending order.'

In [23]:
'''DEPRECATED SOLUTION'''

result_ser = pd.DataFrame(df[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']].apply(lambda x: np.nanmean(x),axis=1))
columns = list(df.index)
result_ser.index = columns
result_ser.sort_values(0,ascending=False,inplace=True)

result_ser.rename(columns={0:"avgGDP"},inplace=True)
result_ser = result_ser.squeeze()
result_ser

United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
Name: avgGDP, dtype: float64

In [24]:
'''MUCH BETTER SOLUTION
MUCH MUCHMUCHMUCHMUCHMUCHMUCHMUCHMUCHMUCHMUCHMUCHMUCH BETTER SOLUTION
'''

def average_in(frame):
    frame["avgGDP"] = np.nanmean(frame[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']])
    
    return frame["avgGDP"]

gdp.groupby("Country").apply(average_in)
#MULTI INDEX???

Country                
Afghanistan         2      1.610842e+10
Albania             4      1.177673e+10
Algeria             57     1.177673e+10
American Samoa      9      1.177673e+10
Andorra             1      1.177673e+10
                               ...     
West Bank and Gaza  256    1.177673e+10
World               257    1.177673e+10
Yemen, Rep.         259    1.177673e+10
Zambia              262    1.177673e+10
Zimbabwe            263    1.177673e+10
Name: avgGDP, Length: 264, dtype: float64


# Question 4

In [25]:
'''By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

This function should return a single number.'''

'By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?\n\nThis function should return a single number.'

In [26]:
# set GDP's index to be the country, since  we already calculated averages we can skip the discovery step
# retrieve the GDP values for 2015 and substract the GDP value for 2006


df.loc["United Kingdom"].loc["2015"] - df.loc["United Kingdom"].loc["2006"]

246702696075.3999

# Question 5

In [27]:
'''What is the mean energy supply per capita?

This function should return a single number.'''

'What is the mean energy supply per capita?\n\nThis function should return a single number.'

In [30]:
# Retrieve the relevant column as a series
# then apply np.average
ser = df["Energy Supply per Capita(GJ)"]
np.average(ser)

157.6

# Question 6

In [29]:
'''What country has the maximum % Renewable and what is the percentage?

This function should return a tuple with the name of the country and the percentage.'''

'What country has the maximum % Renewable and what is the percentage?\n\nThis function should return a tuple with the name of the country and the percentage.'

In [30]:
# creating a copy of the DF with sorted values
df_2 = df.sort_values("% Renewable", ascending=False)

# since the DF is sorted, retrieve the 1st row (this is a series) and from the series retrieve the name
# repeat the indexing and retrieve required row
country_and_renewable = (df_2.iloc[0].name, df_2.iloc[0].loc["% Renewable"])
country_and_renewable

('Brazil', 69.64803)

# Question 7

In [31]:
'''Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?

This function should return a tuple with the name of the country and the ratio.'''

'Question 7\nCreate a new column that is the ratio of Self-Citations to Total Citations. What is the maximum value for this new column, and what country has the highest ratio?\n\nThis function should return a tuple with the name of the country and the ratio.'

In [32]:
df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply(GJ),Energy Supply per Capita(GJ),% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [33]:
#Defining a function to apply the division logic
def division(frame):
    
    frame["Ratio"] = frame["Self-citations"] / frame["Citations"]
    
    #return only relevant columns
    return frame[["Ratio","Self-citations","Citations"]]

# apply division function
df_3 = df.apply(division, axis=1)

#sort inplace
df_3.sort_values("Ratio", ascending=False, inplace=True)


# assign values to the tuple
country_and_ratio = (df_3.iloc[0].name, df_3.iloc[0].loc["Ratio"])
country_and_ratio

('China', 0.6893126179389422)

# Question 8

In [34]:
'''Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?

This function should return the name of the country'''

'Create a column that estimates the population using Energy Supply and Energy Supply per capita. What is the third most populous country according to this estimate?\n\nThis function should return the name of the country'

In [35]:
df

,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply(GJ),Energy Supply per Capita(GJ),% Renewable,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015
Country,,,,,,,,,,,,,,,,,,,,
China,1,127050,126767,597237,411683,4.70,138,1.271910e+11,93,19.754910,3.992331e+12,4.559041e+12,4.997775e+12,5.459247e+12,6.039659e+12,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12
United States,2,96661,94747,792274,265436,8.20,230,9.083800e+10,286,11.570980,1.479230e+13,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13
Japan,3,30504,30287,223024,61554,7.31,134,1.898400e+10,149,10.232820,5.496542e+12,5.617036e+12,5.558527e+12,5.251308e+12,5.498718e+12,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12
United Kingdom,4,20944,20357,206091,37874,9.84,139,7.920000e+09,124,10.600470,2.419631e+12,2.482203e+12,2.470614e+12,2.367048e+12,2.403504e+12,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12
Russian Federation,5,18534,18301,34266,12422,1.85,57,3.070900e+10,214,17.288680,1.385793e+12,1.504071e+12,1.583004e+12,1.459199e+12,1.524917e+12,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12
Canada,6,17899,17620,215003,40930,12.01,149,1.043100e+10,296,61.945430,1.564469e+12,1.596740e+12,1.612713e+12,1.565145e+12,1.613406e+12,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12
Germany,7,17027,16831,140566,27426,8.26,126,1.326100e+10,165,17.901530,3.332891e+12,3.441561e+12,3.478809e+12,3.283340e+12,3.417298e+12,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12
India,8,15005,14841,128763,37209,8.58,115,3.319500e+10,26,14.969080,1.265894e+12,1.374865e+12,1.428361e+12,1.549483e+12,1.708459e+12,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12
France,9,13153,12973,130632,28601,9.93,114,1.059700e+10,166,17.020280,2.607840e+12,2.669424e+12,2.674637e+12,2.595967e+12,2.646995e+12,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12


In [36]:
# create a function to calculate the population

def pop(frame):
    
    frame["Population"] = frame["Energy Supply(GJ)"] / frame["Energy Supply per Capita(GJ)"]
    
    return frame


# update df, and apply the function

df = df.apply(pop, axis=1)

# sort the copy by population, inplace
df.sort_values("Population", ascending=False, inplace=True)

# retrieve the name of the third country

df.iloc[2].name

'United States'

# Question 9 

In [37]:
'''Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).

This function should return a single number.

(Optional: Use the built-in function plot9() to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)'''

"Create a column that estimates the number of citable documents per person. \nWhat is the correlation between the number of citable documents per capita and the energy supply per capita? Use the .corr() method, (Pearson's correlation).\n\nThis function should return a single number.\n\n(Optional: Use the built-in function plot9() to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)"

In [38]:
# define a function to process the new column
def docspercapita(frame):
    frame["Docspcapita"] = frame["Citable documents"] / frame["Population"]
    
    return frame

# apply the function to create the new column
df = df.apply(docspercapita, axis=1)

# find correlation. This accepts the whole column as argument
import scipy.stats as stats

results = stats.pearsonr(df["Docspcapita"], df["Energy Supply per Capita(GJ)"])
results[0]

0.7940010435442943

# Question 10

In [39]:
'''Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, 
and a 0 if the country's % Renewable value is below the median.

This function should return a series named HighRenew whose index is the country name sorted in ascending order of rank.'''

"Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, \nand a 0 if the country's % Renewable value is below the median.\n\nThis function should return a series named HighRenew whose index is the country name sorted in ascending order of rank."

In [31]:
# finding the median
median = np.median(df["% Renewable"])

# applying a lambda to create the new column
df["Larger_than_Median"] = df["% Renewable"].apply(lambda x: x>median)

# converting T/F to Integers
df["Larger_than_Median"] = df["Larger_than_Median"].astype(int)

# finding the median
median = np.median(df["% Renewable"])

# applying a lambda to create the new column
df["Larger_than_Median"] = df["% Renewable"].apply(lambda x: x>median)

# converting T/F to Integers
df["Larger_than_Median"] = df["Larger_than_Median"].astype(int)

# sorting before cutting
df= df.sort_values("Rank", ascending= True)

# creating the requested series and sorting.
HighRenew = df["Rank"]
HighRenew

Country
China                  1
United States          2
Japan                  3
United Kingdom         4
Russian Federation     5
Canada                 6
Germany                7
India                  8
France                 9
South Korea           10
Italy                 11
Spain                 12
Iran                  13
Australia             14
Brazil                15
Name: Rank, dtype: int64

# Question 11

In [41]:
#create a new column that links the existing dictionary to countries and continents
cont_country = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}


# reset index
df.reset_index(inplace=True)

# create the new column via apply and lambda.
    ## Lambdas searches for the value in the dictionary and if not found returns nan.
df["continent"] = df["Country"].apply(lambda x: cont_country[x] if x in cont_country else np.nan)

# process results
    #we can rename the returning columns by passing 
        ###COLUMN NAME
        ### AND A TUPLE OF THE REFERENCE COLUMN AND AGGREGATION FUNCTION

results = df.groupby("continent").agg(size=("Population",len),
                                      sum=("Population",np.sum),
                                     mean=("Population",np.average),
                                     std=("Population",np.std))

# Question 12

In [42]:
# creating the bins and storing them in a series/df
cut_df = pd.cut(df["% Renewable"],5)

# appending a new column based on bins. The order is preserved
df["% Renewable"] = cut_df
df

,Country,Rank,Documents,Citable documents,Citations,Self-citations,Citations per document,H index,Energy Supply(GJ),Energy Supply per Capita(GJ),...,2011,2012,2013,2014,2015,Population,Docspcapita,Larger_than_Median,continent,% Bins
0,China,1.0,127050.0,126767.0,597237.0,411683.0,4.70,138.0,1.271910e+11,93.0,...,6.612490e+12,7.124978e+12,7.672448e+12,8.230121e+12,8.797999e+12,1.367645e+09,0.000093,1,Asia,"(15.753, 29.227]"
1,United States,2.0,96661.0,94747.0,792274.0,265436.0,8.20,230.0,9.083800e+10,286.0,...,1.520402e+13,1.554216e+13,1.577367e+13,1.615662e+13,1.654857e+13,3.176154e+08,0.000298,0,North America,"(2.212, 15.753]"
2,Japan,3.0,30504.0,30287.0,223024.0,61554.0,7.31,134.0,1.898400e+10,149.0,...,5.473738e+12,5.569102e+12,5.644659e+12,5.642884e+12,5.669563e+12,1.274094e+08,0.000238,0,Asia,"(2.212, 15.753]"
3,United Kingdom,4.0,20944.0,20357.0,206091.0,37874.0,9.84,139.0,7.920000e+09,124.0,...,2.450911e+12,2.479809e+12,2.533370e+12,2.605643e+12,2.666333e+12,6.387097e+07,0.000319,0,Europe,"(2.212, 15.753]"
4,Russian Federation,5.0,18534.0,18301.0,34266.0,12422.0,1.85,57.0,3.070900e+10,214.0,...,1.589943e+12,1.645876e+12,1.666934e+12,1.678709e+12,1.616149e+12,1.435000e+08,0.000128,1,Europe,"(15.753, 29.227]"
5,Canada,6.0,17899.0,17620.0,215003.0,40930.0,12.01,149.0,1.043100e+10,296.0,...,1.664087e+12,1.693133e+12,1.730688e+12,1.773486e+12,1.792609e+12,3.523986e+07,0.000500,1,North America,"(56.174, 69.648]"
6,Germany,7.0,17027.0,16831.0,140566.0,27426.0,8.26,126.0,1.326100e+10,165.0,...,3.542371e+12,3.556724e+12,3.567317e+12,3.624386e+12,3.685556e+12,8.036970e+07,0.000209,1,Europe,"(15.753, 29.227]"
7,India,8.0,15005.0,14841.0,128763.0,37209.0,8.58,115.0,3.319500e+10,26.0,...,1.821872e+12,1.924235e+12,2.051982e+12,2.200617e+12,2.367206e+12,1.276731e+09,0.000012,0,Asia,"(2.212, 15.753]"
8,France,9.0,13153.0,12973.0,130632.0,28601.0,9.93,114.0,1.059700e+10,166.0,...,2.702032e+12,2.706968e+12,2.722567e+12,2.729632e+12,2.761185e+12,6.383735e+07,0.000203,0,Europe,"(15.753, 29.227]"
9,South Korea,10.0,11983.0,11923.0,114675.0,22595.0,9.57,104.0,1.100700e+10,221.0,...,1.134796e+12,1.160809e+12,1.194429e+12,1.234340e+12,1.266580e+12,4.980543e+07,0.000239,0,Asia,"(2.212, 15.753]"


In [44]:
'''SOLUTION VIA PIVOT'''

bins_df2 = df.pivot_table(values="Country", index=["continent","% Renewable"], aggfunc=len)
bins_df2

Country
continent     % Bins                   
Asia          (2.212, 15.753]       4.0
              (15.753, 29.227]      1.0
Australia     (2.212, 15.753]       1.0
Europe        (2.212, 15.753]       1.0
              (15.753, 29.227]      3.0
              (29.227, 42.701]      2.0
North America (2.212, 15.753]       1.0
              (56.174, 69.648]      1.0
South America (56.174, 69.648]      1.0

In [46]:
'''HANDMADE SOLUTION'''

##### a copy
bins_df = df

# setting multilevel index
bins_df = bins_df.set_index(["continent","% Bins"])

# grouping by multilevel index and counting the elements of each bin
bins_df = bins_df.groupby(level=(0,1)).agg({"Country":len})

# drop NAN
bins_df.dropna(inplace=True)

# squeezing into a series
bins_df.squeeze()

continent      % Bins          
Asia           (2.212, 15.753]     4.0
               (15.753, 29.227]    1.0
Australia      (2.212, 15.753]     1.0
Europe         (2.212, 15.753]     1.0
               (15.753, 29.227]    3.0
               (29.227, 42.701]    2.0
North America  (2.212, 15.753]     1.0
               (56.174, 69.648]    1.0
South America  (56.174, 69.648]    1.0
Name: Country, dtype: float64

# Question 13

In [47]:
# found in stackoverflow -> ADD THOUSANDS SEPARATOR PYTHON --> {0:,}'.format(#######)

#applying to create a new column
df["PopEst"] = df["Population"].apply(lambda x: '{0:,}'.format(x))

# setting index to country again
df = df.set_index("Country")

#Retrieve the relevant series
popest_df = df["PopEst"]
popest_df

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object